# Llama-Index Knowledge Based 
[tutorial](https://betterprogramming.pub/how-to-build-your-own-custom-chatgpt-with-custom-knowledge-base-4e61ad82427e)


## Tree Index

In [14]:
import os
from dotenv import load_dotenv
import logging
import sys

load_dotenv()
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [24]:
from llama_index import GPTTreeIndex, SimpleDirectoryReader, StorageContext
from IPython.display import Markdown, display
storage_context = StorageContext.from_defaults()
persist_dir = "./tree"

documents = SimpleDirectoryReader('data').load_data()
new_index = GPTTreeIndex.from_documents(documents, storage_context=storage_context)
storage_context.persist(persist_dir)

INFO:llama_index.indices.common_tree.base:> Building index from nodes: 2 chunks
> Building index from nodes: 2 chunks
> Building index from nodes: 2 chunks
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 10701 tokens
> [build_index_from_nodes] Total LLM token usage: 10701 tokens
> [build_index_from_nodes] Total LLM token usage: 10701 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 0 tokens
> [build_index_from_nodes] Total embedding token usage: 0 tokens
> [build_index_from_nodes] Total embedding token usage: 0 tokens


In [25]:
query_engine = new_index.as_query_engine()
response = query_engine.query("Are all sugar-free products calorie-free?")
display(Markdown(f"<b>{response}</b>"))

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer')).
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer')).
INFO:llama_index.indices.tree.select_leaf_retriever:>[Level 0] Selected node: [3]/[3]
>[Level 0] Selected node: [3]/[3]
>[Level 0] Selected node: [3]/[3]
INFO:llama_index.indices.tree.select_leaf_retriever:>[Level 1] Selected node: [7]/[7]
>[Level 1] Selected node: [7]/[7]
>[Level 1] Selected node: [7]/[7]
INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 4333 tokens
> [retrieve] Total LLM token usage: 4333 tokens
> [retrieve] Total LLM token usage: 4333 tokens
INFO:llama_

<b>
No, not all sugar-free products are calorie-free. Sugar-free products may still contain calories from other sources such as fat, protein, and carbohydrates. It is important to read the nutrition label to check the calorie content of the product.</b>

In [27]:
response.response

'\nNo, not all sugar-free products are calorie-free. Sugar-free products may still contain calories from other sources such as fat, protein, and carbohydrates. It is important to read the nutrition label to check the calorie content of the product.'

## Save Index

In [20]:
from llama_index import GPTVectorStoreIndex
persist_dir = "./persist"
vector_store_index = GPTVectorStoreIndex.from_documents(documents, storage_context=storage_context)
storage_context.persist(persist_dir)

INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
> [build_index_from_nodes] Total LLM token usage: 0 tokens
> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 23443 tokens
> [build_index_from_nodes] Total embedding token usage: 23443 tokens
> [build_index_from_nodes] Total embedding token usage: 23443 tokens


In [22]:
query_engine = vector_store_index.as_query_engine()
response = query_engine.query("If I have diabetes, does that mean I can never consume starchy foods?")
display(Markdown(f"<b>{response}</b>"))

INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 16 tokens
> [retrieve] Total embedding token usage: 16 tokens
> [retrieve] Total embedding token usage: 16 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 1818 tokens
> [get_response] Total LLM token usage: 1818 tokens
> [get_response] Total LLM token usage: 1818 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens
> [get_response] Total embedding token usage: 0 tokens
> [get_response] Total embedding token usage: 0 tokens


<b>
No. Carbohydrate foods, particularly starchy foods such as rice, bread, noodles and cereals, form a major component of the body's energy source. All starchy foods</b>

## Vector Store Index

In [2]:
from llama_index import GPTVectorStoreIndex, StorageContext, SimpleDirectoryReader
import os
import logging
import sys
from dotenv import load_dotenv
from IPython.display import display, Markdown

load_dotenv()
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [14]:
logging.disable()

In [3]:
from llama_index.retrievers import VectorIndexRetriever
from llama_index.indices.postprocessor import SimilarityPostprocessor
from llama_index.query_engine import RetrieverQueryEngine
from llama_index import ResponseSynthesizer, load_index_from_storage

storage_context = StorageContext.from_defaults(persist_dir="./vector_store")

In [104]:
documents = SimpleDirectoryReader('data', exclude=['raw_data_combined.txt']).load_data()
docs_head = documents[:50]
docs_tail = documents[50:]
# vector_store_index = GPTVectorStoreIndex.from_documents(docs_head, storage_context=storage_context)

In [105]:
vector_store_index = load_index_from_storage(storage_context=storage_context)

In [66]:
retriever = VectorIndexRetriever(
    index=vector_store_index,
    similarity_top_k=2,
)

response_synthesizer = ResponseSynthesizer.from_args(
    node_postprocessors=[
        SimilarityPostprocessor(similarity_cutoff=0.85)
    ]
)

query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer
)

response = query_engine.query('How much carb and sugar shoul I eat?')
display(Markdown(f"<b>{response}</b>"))
[display(Markdown(f"{s.node.get_text()} \n similarity: {s.score}")) for s in response.source_nodes]

<b>
It is difficult to give a definitive answer to this question without knowing more about your individual health and dietary needs. It is best to speak to a Dietician to determine how much carbohydrate and sugar you should be consuming.</b>

/home/hedgehog/softwares/anaconda3/lib/python3.9/site-packages/llama_index/data_structs/node.py:191: UserWarning: .similarity is deprecated, use .score instead instead
  warnings.warn(".similarity is deprecated, use .score instead instead")


question: If it is sugar-free, I can eat as much as I want, right?
answer: Foods with ‘Sugar-free' or ‘No sugar added' labels do not mean that the food is carbohydrate-free. Carbohydrates is the nutrient that has the greatest effect on your blood glucose levels.
Source: Win Against Diabetes Booklet.
 
 similarity: 0.8636490243002047

question: If I have diabetes, does that mean I can never consume sugar?
answer: You may still be able to include some sugar in your diet by learning carbohydrate counting and knowing your requirements. Speak to your Dietician to know how.
 
 similarity: 0.8535845911727508

[None, None]

### Cohere Rerank

In [22]:
import os
from dotenv import load_dotenv
from llama_index.indices.postprocessor.cohere_rerank import CohereRerank
from llama_index import GPTVectorStoreIndex, SimpleDirectoryReader

load_dotenv()
api_key = os.environ["COHERE_API_KEY"]
cohere_rerank = CohereRerank(api_key=api_key, top_n=2)

In [67]:
retriever = VectorIndexRetriever(
    index=vector_store_index,
    similarity_top_k=10,
)

response_synthesizer = ResponseSynthesizer.from_args(
    node_postprocessors=[
        SimilarityPostprocessor(similarity_cutoff=0.85),
        CohereRerank(api_key=api_key, top_n=2),
    ]
)

query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

response = query_engine.query('How much carb and sugar shoul I eat?')
display(Markdown(f"<b>{response}</b>"))
[display(Markdown(f"{s.node.get_text()} \n score: {s.similarity}")) for s in response.source_nodes]

[nltk_data] Downloading package punkt to /home/hedgehog/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


TypeError: __init__() got an unexpected keyword argument 'optimizer'

### Optimizer

In [106]:
from llama_index.optimization.optimizer import SentenceEmbeddingOptimizer
from llama_index import LLMPredictor, GPTVectorStoreIndex, PromptHelper, ServiceContext
from langchain import OpenAI

llm_predictor = LLMPredictor(llm=OpenAI(temperature=0, model_name="text-curie-001"))
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor)
# optimizer = SentenceEmbeddingOptimizer(threshold_cutoff=0.6)
storage_context_default = StorageContext.from_defaults()

index = GPTVectorStoreIndex.from_documents(documents=documents, storage_context=storage_context_default, service_context=service_context)

opt_query_engine = vector_store_index.as_query_engine(
    retriever=VectorIndexRetriever(
        index=index,
        similarity_top_k=2,
    ),
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.85)],
    # optimizer=optimizer,
)

opt_query_engine.query(test_queries[0])
print(response.response)


The correct answer is:

The correct answer is:

The correct answer is:

The correct answer is:

The correct answer is:

The correct answer is:

The correct answer is:

The correct answer is:

The correct answer is:

The correct answer is:

The correct answer is:

The correct answer is:

The correct answer is:

The correct answer is:

The correct answer is:

The correct answer is:

The correct answer is:

The correct answer is:

The correct answer is:

The correct answer is:

The correct answer is:

The correct answer is:

The correct answer is:

The correct answer is:

The correct answer is:

The correct answer is:

The correct answer is:

The correct answer is:

The correct answer is:

The correct answer is:

The correct answer is:

The correct answer is:

The correct answer is:

The correct answer is:

The correct answer is:

The correct answer is:

The correct answer


In [107]:
response = opt_query_engine.query(test_queries[0])
print(response.response)


No, switching to wholegrain rice does not mean you can eat more rice. The carbohydrate content of wholegrain rice is similar to regular rice, but it provides more fibre, making it a better choice for diabetes. Eating too much of any type of carbohydrates, including wholegrains, can still cause poor blood sugar control, so moderation is key.


### Single-step decomposition
Not Yet

## Evaluation

### Preparation

In [58]:
import logging
import sys
import time
from typing import List

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index import (
    GPTTreeIndex, 
    GPTVectorStoreIndex, 
    SimpleDirectoryReader, 
    LLMPredictor, 
    ServiceContext,
    Response
)
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from llama_index.evaluation import ResponseEvaluator
import pandas as pd
pd.set_option('display.max_colwidth', 0)

In [7]:
llm_predictor = LLMPredictor(llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo"))
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor)
evaluator = ResponseEvaluator(service_context=service_context)

Unknown max input size for gpt-3.5-turbo, using defaults.
Unknown max input size for gpt-3.5-turbo, using defaults.


In [71]:
def eval_df(query: str, response: Response, eval_result: str, time: float):
    eval_df = pd.DataFrame(
        {
            "Query": query,
            "Response": str(response), 
            "Source": response.source_nodes[0].node.get_text()[:1000] + "...",
            "Time": time,
            "Evaluation Result": eval_result
        },
        index=[0]
    )
    return eval_df

def eval_source(query: str, response: Response, eval_result: List[str]):
    sources = [s.node.get_text() for s in response.source_nodes]
    eval_df = pd.DataFrame(
        {
            "Query": query,
            "Source": sources,
            "Eval Result": eval_result, 
        }
    )
    return eval_df


In [10]:
test_queries = [
    # original questions
    'Does switching to wholegrain rice mean I can eat more rice?',
    'Do I need to follow a low carb diet?',
    'Can we store glucose in our body?',
    # modified questions
    'Will eating less carbohydrate benefit my body?',
    'How can I eat healthily?',
    'Will I be able to have sugar as a diabetics?',
    # combined knowledge
    'Is a diet high in sugar and carb bad for me?',
    'How much carb and sugar shoul I eat?',
    'What are the types of diabetes and how to cure them?'
]

### Check if the sources (context) contains the infomation in the response

In [110]:
responses = []
times = []
for q in test_queries:
    start_time = time.time()
    response = query_engine.query(q)
    end_time = time.time()
    responses.append(response)
    times.append(end_time - start_time)
eval_results = [evaluator.evaluate(res) for res in responses]
eval_vects = [eval_df(test_queries[i], responses[i], eval_results[i], times[i]) for i in range(len(responses))]
eval_tbl = pd.concat(eval_vects, ignore_index=True)
display(eval_tbl)

/home/hedgehog/softwares/anaconda3/lib/python3.9/site-packages/llama_index/data_structs/node.py:176: UserWarning: .source_text is deprecated, use .node.get_text() instead
  warnings.warn(".source_text is deprecated, use .node.get_text() instead")


,Query,Response,Source,Time,Evaluation Result
0,Does switching to wholegrain rice mean I can eat more rice?,"\nNo, switching to wholegrain rice does not mean you can eat more rice. The carbohydrate content of wholegrain rice is similar to regular rice, but it provides more fibre, making it a better choice for diabetes. Eating too much of any type of carbohydrates, including wholegrains, can still cause poor blood sugar control, so moderation is key.","question: Does switching to wholegrain rice mean I can eat more rice?\nanswer: The carbohydrate content of wholegrain rice is actually similar to regular rice. Still, wholegrain rice provides more fibre by comparison, hence making it a better choice for diabetes.\n...",5.580032,YES
1,Do I need to follow a low carb diet?,"\nNo, you do not need to follow a low carb diet. However, it is important to understand how different foods affect your blood glucose levels, especially carbohydrates, and to keep track of the amount of carbohydrates you consume. When choosing carbohydrates, it is recommended to pick starchy carbohydrates that release glucose into the blood stream at a slower rate.","question: Do I need to follow a low carb diet?\nanswer: People with diabetes do not need to go on a special diet. However, it is important that you understand how different foods affect your blood glucose levels, especially carbohydrates, since it the nutrient that has the greatest effect on your blood glucose levels. An important step to effective manage diabetes nutrition is to keep track of the amount of carbohydrates you consume by using carbohydrates counting techniques. When choosing the types of carbohydrates, pick starchy carbohydrate such as rice, noodles, pasta, cereals and starchy vegetables such as potatoes as these carbs release glucose into the blood stream at a slower rate compared to sugars, thus promoting a more stable blood glucose level.\nSource: Win Against Diabetes Booklet.\n...",5.469756,NO
2,Can we store glucose in our body?,"\nYes, glucose can be stored in our body as glycogen mainly in the liver, muscles, and fat tissues.","question: Can we store glucose in our body?\nanswer: Yes, glucose is stored as glycogen mainly in the liver, muscles, and fat tissues. Our body knows that glucose is a good product, and the kidneys act as filters to reabsorb the glucose into our blood by coating the red blood cells and transferring them to the muscle and liver to be stored as glycogen. When the liver, the main store, is full, excess glucose will be stored as triglycerides in the fat tissues. When our body needs energy, the stored glucose and glycogen in the liver will be broken down by insulin and converted to energy.\nSource: Red Ants Crawling to My Urine.\n...",3.199424,NO
3,Will eating less carbohydrate benefit my body?,"\nYes, eating less carbohydrate can benefit your body. Eating a balanced diet with the right amount of carbohydrates, proteins, and fats can help you maintain a healthy weight, provide energy, and support overall health.","question: Can I eat as much as I want, as long as I cut down on my carbohydrate intake?\nanswer: Excessive intake of any nutrient, including protein, may lead to weight gain. It would be advisable to know your body's requirements for all nutrients and keep to them.\n...",3.123333,NO
4,How can I eat healthily?,"\nTo eat healthily, you should aim to have a well-balanced diet that consists of less sugar, less fat, less salt, and more fibre. Incorporate more fresh fruits and vegetables, wholegrains, and lean meats into your meals. Consider replacing sweetened beverages, sweets and cakes with milk, fruit and wholemeal crackers. Limit intake of fried dishes and use less butter, ghee and coconut oil in cooking. Use low fat cooking methods such as baking, boiling, grilling and steaming. Use less preserved food such as salted fish and canned meats, and more fresh products. Switch to unpolished brown rice, wholegrain cereals and brains, and include fresh fruit 

In [108]:
responses2 = []
times2 = []
for q in test_queries:
    start_time = time.time()
    response = opt_query_engine.query(q)
    end_time = time.time()
    responses2.append(response)
    times2.append(end_time - start_time)
eval_results2 = [evaluator.evaluate(res) for res in responses2]
eval_vects2 = [eval_df(test_queries[i], responses2[i], eval_results[i], times2[i]) for i in range(len(responses2))]
eval_tbl2 = pd.concat(eval_vects2, ignore_index=True)
display(eval_tbl2)

/home/hedgehog/softwares/anaconda3/lib/python3.9/site-packages/llama_index/data_structs/node.py:176: UserWarning: .source_text is deprecated, use .node.get_text() instead
  warnings.warn(".source_text is deprecated, use .node.get_text() instead")


,Query,Response,Source,Time,Evaluation Result
0,Does switching to wholegrain rice mean I can eat more rice?,"\nNo, switching to wholegrain rice does not mean you can eat more rice. The carbohydrate content of wholegrain rice is similar to regular rice, but it provides more fibre, making it a better choice for diabetes. Eating too much of any type of carbohydrates, including wholegrains, can still cause poor blood sugar control, so moderation is key.","question: Does switching to wholegrain rice mean I can eat more rice?\nanswer: The carbohydrate content of wholegrain rice is actually similar to regular rice. Still, wholegrain rice provides more fibre by comparison, hence making it a better choice for diabetes.\n...",5.733148,YES
1,Do I need to follow a low carb diet?,"\nNo, you do not need to follow a low carb diet. However, it is important to understand how different foods affect your blood glucose levels, especially carbohydrates, and to keep track of the amount of carbohydrates you consume. When choosing carbohydrates, it is recommended to pick starchy carbohydrates that release glucose into the blood stream at a slower rate.","question: Do I need to follow a low carb diet?\nanswer: People with diabetes do not need to go on a special diet. However, it is important that you understand how different foods affect your blood glucose levels, especially carbohydrates, since it the nutrient that has the greatest effect on your blood glucose levels. An important step to effective manage diabetes nutrition is to keep track of the amount of carbohydrates you consume by using carbohydrates counting techniques. When choosing the types of carbohydrates, pick starchy carbohydrate such as rice, noodles, pasta, cereals and starchy vegetables such as potatoes as these carbs release glucose into the blood stream at a slower rate compared to sugars, thus promoting a more stable blood glucose level.\nSource: Win Against Diabetes Booklet.\n...",5.362114,NO
2,Can we store glucose in our body?,"\nYes, glucose can be stored in our body as glycogen mainly in the liver, muscles, and fat tissues.","question: Can we store glucose in our body?\nanswer: Yes, glucose is stored as glycogen mainly in the liver, muscles, and fat tissues. Our body knows that glucose is a good product, and the kidneys act as filters to reabsorb the glucose into our blood by coating the red blood cells and transferring them to the muscle and liver to be stored as glycogen. When the liver, the main store, is full, excess glucose will be stored as triglycerides in the fat tissues. When our body needs energy, the stored glucose and glycogen in the liver will be broken down by insulin and converted to energy.\nSource: Red Ants Crawling to My Urine.\n...",3.682570,NO
3,Will eating less carbohydrate benefit my body?,"\nYes, eating less carbohydrate can benefit your body. Eating a balanced diet with the right amount of carbohydrates, proteins, and fats can help you maintain a healthy weight, provide energy, and support overall health.","question: Can I eat as much as I want, as long as I cut down on my carbohydrate intake?\nanswer: Excessive intake of any nutrient, including protein, may lead to weight gain. It would be advisable to know your body's requirements for all nutrients and keep to them.\n...",2.966472,NO
4,How can I eat healthily?,"\nTo eat healthily, you should aim to have a well-balanced diet that consists of less sugar, less fat, less salt, and more fibre. You can do this by replacing sweetened beverages, sweets and cakes with milk, fruit and wholemeal crackers, limiting intake of fried dishes and using less butter, ghee and coconut oil in cooking, using less preserved food such as salted fish and canned meats, and more fresh products, and switching to unpolished brown rice, wholegrain cereals and brains, and including fresh fruit and vegetables. Additionally, you should follow Health Promotion Board's My Healthy Plate guidelines and fill half your plate with fruits and

In [111]:
compare_tbl = pd.concat([eval_tbl, eval_tbl2], axis=1)
display(compare_tbl)

,Query,Response,Source,Time,Evaluation Result,Query,Response,Source,Time,Evaluation Result
0,Does switching to wholegrain rice mean I can eat more rice?,"\nNo, switching to wholegrain rice does not mean you can eat more rice. The carbohydrate content of wholegrain rice is similar to regular rice, but it provides more fibre, making it a better choice for diabetes. Eating too much of any type of carbohydrates, including wholegrains, can still cause poor blood sugar control, so moderation is key.","question: Does switching to wholegrain rice mean I can eat more rice?\nanswer: The carbohydrate content of wholegrain rice is actually similar to regular rice. Still, wholegrain rice provides more fibre by comparison, hence making it a better choice for diabetes.\n...",5.580032,YES,Does switching to wholegrain rice mean I can eat more rice?,"\nNo, switching to wholegrain rice does not mean you can eat more rice. The carbohydrate content of wholegrain rice is similar to regular rice, but it provides more fibre, making it a better choice for diabetes. Eating too much of any type of carbohydrates, including wholegrains, can still cause poor blood sugar control, so moderation is key.","question: Does switching to wholegrain rice mean I can eat more rice?\nanswer: The carbohydrate content of wholegrain rice is actually similar to regular rice. Still, wholegrain rice provides more fibre by comparison, hence making it a better choice for diabetes.\n...",5.733148,YES
1,Do I need to follow a low carb diet?,"\nNo, you do not need to follow a low carb diet. However, it is important to understand how different foods affect your blood glucose levels, especially carbohydrates, and to keep track of the amount of carbohydrates you consume. When choosing carbohydrates, it is recommended to pick starchy carbohydrates that release glucose into the blood stream at a slower rate.","question: Do I need to follow a low carb diet?\nanswer: People with diabetes do not need to go on a special diet. However, it is important that you understand how different foods affect your blood glucose levels, especially carbohydrates, since it the nutrient that has the greatest effect on your blood glucose levels. An important step to effective manage diabetes nutrition is to keep track of the amount of carbohydrates you consume by using carbohydrates counting techniques. When choosing the types of carbohydrates, pick starchy carbohydrate such as rice, noodles, pasta, cereals and starchy vegetables such as potatoes as these carbs release glucose into the blood stream at a slower rate compared to sugars, thus promoting a more stable blood glucose level.\nSource: Win Against Diabetes Booklet.\n...",5.469756,NO,Do I need to follow a low carb diet?,"\nNo, you do not need to follow a low carb diet. However, it is important to understand how different foods affect your blood glucose levels, especially carbohydrates, and to keep track of the amount of carbohydrates you consume. When choosing carbohydrates, it is recommended to pick starchy carbohydrates that release glucose into the blood stream at a slower rate.","question: Do I need to follow a low carb diet?\nanswer: People with diabetes do not need to go on a special diet. However, it is important that you understand how different foods affect your blood glucose levels, especially carbohydrates, since it the nutrient that has the greatest effect on your blood glucose levels. An important step to effective manage diabetes nutrition is to keep track of the amount of carbohydrates you consume by using carbohydrates counting techniques. When choosing the types of carbohydrates, pick starchy carbohydrate such as rice, noodles, pasta, cereals and starchy vegetables such as potatoes as these carbs release glucose into the blood stream at a slower rate compared to sugars, thus promoting a more stable blood glucose level.\nSource: Win Against Diabetes Booklet.\n...",5.362114,NO
2,Can we store glucose in our body?,"\nYes, glucose can be 

compare_df = pd.concat([])

### Check if the sources (context) retrieved are relevant to the queries

In [113]:
from llama_index.evaluation import QueryResponseEvaluator
query_evaluator = QueryResponseEvaluator(service_context=service_context)
eval_source_results = [query_evaluator.evaluate_source_nodes(test_queries[i], responses[i]) for i in range(len(responses))]
eval_source_vects = [eval_source(test_queries[i], responses[i], eval_source_results[i]) for i in range(len(responses))]
display(pd.concat(eval_source_vects, ignore_index=True))

/home/hedgehog/softwares/anaconda3/lib/python3.9/site-packages/llama_index/data_structs/node.py:176: UserWarning: .source_text is deprecated, use .node.get_text() instead
  warnings.warn(".source_text is deprecated, use .node.get_text() instead")


,Query,Source,Eval Result
0,Does switching to wholegrain rice mean I can eat more rice?,"question: Does switching to wholegrain rice mean I can eat more rice?\nanswer: The carbohydrate content of wholegrain rice is actually similar to regular rice. Still, wholegrain rice provides more fibre by comparison, hence making it a better choice for diabetes.\n",YES
1,Does switching to wholegrain rice mean I can eat more rice?,"question: Does white rice cause diabetes?\nanswer: Although recent studies by the Harvard School of Public Health and the National University Health System found that white rice has a high Glycaemic Index (GI), it is important to understand its overall health impact when taken as a part of our daily meals. GI ranks carbohydrates in food based on the rate at which they affect blood sugar levels. The important difference between sugar and rice is that sugar contributes calories with little nutritional benefit, while rice provides satiety and can be part of a healthy, balanced diet if accompanied by side dishes packed with nutritional value such as tofu, fish, lean meat, eggs and vegetables. In contrast, it is easy to eat too much sugar as it does not give the feeling of fullness and can lead to obesity, raising the risk of type 2 diabetes. Wholegrains like brown or red rice are better than white rice as these contain more nutrients like iron, magnesium, B vitamins and fibre. Wholegrains also take more time to digest, giving the feeling of fullness for longer. However, do note that any type of carbohydrates, including wholegrains, when taken in excess can still cause poor blood sugar control. Moderation is key.\n",YES
2,Do I need to follow a low carb diet?,"question: Do I need to follow a low carb diet?\nanswer: People with diabetes do not need to go on a special diet. However, it is important that you understand how different foods affect your blood glucose levels, especially carbohydrates, since it the nutrient that has the greatest effect on your blood glucose levels. An important step to effective manage diabetes nutrition is to keep track of the amount of carbohydrates you consume by using carbohydrates counting techniques. When choosing the types of carbohydrates, pick starchy carbohydrate such as rice, noodles, pasta, cereals and starchy vegetables such as potatoes as these carbs release glucose into the blood stream at a slower rate compared to sugars, thus promoting a more stable blood glucose level.\nSource: Win Against Diabetes Booklet.\n",YES
3,Can we store glucose in our body?,"question: Can we store glucose in our body?\nanswer: Yes, glucose is stored as glycogen mainly in the liver, muscles, and fat tissues. Our body knows that glucose is a good product, and the kidneys act as filters to reabsorb the glucose into our blood by coating the red blood cells and transferring them to the muscle and liver to be stored as glycogen. When the liver, the main store, is full, excess glucose will be stored as triglycerides in the fat tissues. When our body needs energy, the stored glucose and glycogen in the liver will be broken down by insulin and converted to energy.\nSource: Red Ants Crawling to My Urine.\n",YES
4,Can we store glucose in our body?,question: What is glucose?\nanswer: Glucose comes from our food and is also produced by the liver. It is the main source of energy for the body's cells and is transported through the bloodstream. Glucose needs to enter cells to be used.\n,YES
5,Will eating less carbohydrate benefit my body?,"question: Can I eat as much as I want, as long as I cut down on my carbohydrate intake?\nanswer: Excessive intake of any nutrient, including protein, may lead to weight gain. It would be advisable to know your body's requirements for all nutrients and keep to them.\n",NO
6,How can I eat healthily?,"question: What are some strategies I can use to have a well-balanced diet?\nanswer: To consume less sugary foods, consider replacing sweetened beverages, sweets and cakes, with milk, f

### Check if response answers the question

In [17]:
eval_qr_results = [query_evaluator.evaluate(test_queries[i], responses[i]) for i in range(len(responses))]
eval_qr_vects = [eval_df(test_queries[i], responses[i], eval_qr_results[i]) for i in range(len(responses))]
display(pd.concat(eval_source_vects, ignore_index=True))

,Query,Source,Eval Result
0,Does switching to wholegrain rice mean I can eat more rice?,"question: Does switching to wholegrain rice mean I can eat more rice?\nanswer: The carbohydrate content of wholegrain rice is actually similar to regular rice. Still, wholegrain rice provides more fibre by comparison, hence making it a better choice for diabetes.\n",NO
1,Does switching to wholegrain rice mean I can eat more rice?,"question: Does white rice cause diabetes?\nanswer: Although recent studies by the Harvard School of Public Health and the National University Health System found that white rice has a high Glycaemic Index (GI), it is important to understand its overall health impact when taken as a part of our daily meals. GI ranks carbohydrates in food based on the rate at which they affect blood sugar levels. The important difference between sugar and rice is that sugar contributes calories with little nutritional benefit, while rice provides satiety and can be part of a healthy, balanced diet if accompanied by side dishes packed with nutritional value such as tofu, fish, lean meat, eggs and vegetables. In contrast, it is easy to eat too much sugar as it does not give the feeling of fullness and can lead to obesity, raising the risk of type 2 diabetes. Wholegrains like brown or red rice are better than white rice as these contain more nutrients like iron, magnesium, B vitamins and fibre. Wholegrains also take more time to digest, giving the feeling of fullness for longer. However, do note that any type of carbohydrates, including wholegrains, when taken in excess can still cause poor blood sugar control. Moderation is key.\n",YES
2,Do I need to follow a low carb diet?,"question: Do I need to follow a low carb diet?\nanswer: People with diabetes do not need to go on a special diet. However, it is important that you understand how different foods affect your blood glucose levels, especially carbohydrates, since it the nutrient that has the greatest effect on your blood glucose levels. An important step to effective manage diabetes nutrition is to keep track of the amount of carbohydrates you consume by using carbohydrates counting techniques. When choosing the types of carbohydrates, pick starchy carbohydrate such as rice, noodles, pasta, cereals and starchy vegetables such as potatoes as these carbs release glucose into the blood stream at a slower rate compared to sugars, thus promoting a more stable blood glucose level.\nSource: Win Against Diabetes Booklet.\n",YES
3,Can we store glucose in our body?,"question: Can we store glucose in our body?\nanswer: Yes, glucose is stored as glycogen mainly in the liver, muscles, and fat tissues. Our body knows that glucose is a good product, and the kidneys act as filters to reabsorb the glucose into our blood by coating the red blood cells and transferring them to the muscle and liver to be stored as glycogen. When the liver, the main store, is full, excess glucose will be stored as triglycerides in the fat tissues. When our body needs energy, the stored glucose and glycogen in the liver will be broken down by insulin and converted to energy.\nSource: Red Ants Crawling to My Urine.\n",YES
4,Can we store glucose in our body?,question: What is glucose?\nanswer: Glucose comes from our food and is also produced by the liver. It is the main source of energy for the body's cells and is transported through the bloodstream. Glucose needs to enter cells to be used.\n,YES
5,Will eating less carbohydrate benefit my body?,"question: Can I eat as much as I want, as long as I cut down on my carbohydrate intake?\nanswer: Excessive intake of any nutrient, including protein, may lead to weight gain. It would be advisable to know your body's requirements for all nutrients and keep to them.\n",YES
6,How can I eat healthily?,"question: What are some strategies I can use to have a well-balanced diet?\nanswer: To consume less sugary foods, consider replacing sweetened beverages, sweets and cakes, with milk, f